In [0]:

CREATE OR REFRESH STREAMING TABLE newpavancatalog.bronze.bronze_yellow_taxi
COMMENT "Raw orders data ingested from the source system"
TBLPROPERTIES ("quality" = "bronze")
AS
SELECT *,
        _metadata.file_path AS input_file_path,
       CURRENT_TIMESTAMP AS ingestion_timestamp
  FROM cloud_files(
                    '/Volumes/newpavancatalog/bronze/nyc_taxi_data_volume', 
                    'csv',
                    map("cloudFiles.inferColumnTypes", "true")
                   );


In [0]:
-- Then recreate it with your current definition
CREATE OR REFRESH STREAMING TABLE silver_yellowTaxi
COMMENT "Deduplicated and cleaned Yellow Taxi trip data"
TBLPROPERTIES ("quality" = "silver")
AS
SELECT DISTINCT *
FROM STREAM(LIVE.bronze_yellow_taxi)
WHERE 
  VendorID IS NOT NULL
  AND tpep_pickup_datetime IS NOT NULL
  AND tpep_dropoff_datetime IS NOT NULL
  AND passenger_count IS NOT NULL
  AND trip_distance IS NOT NULL
  AND fare_amount IS NOT NULL
  AND total_amount IS NOT NULL
  AND tip_amount IS NOT NULL;

In [0]:
CREATE OR REFRESH STREAMING TABLE gold_yellowTaxi
COMMENT "Aggregated Yellow Taxi trip metrics by pickup datetime and vendor"
TBLPROPERTIES ("quality" = "gold")
AS
SELECT 
  tpep_pickup_datetime AS Pickup_Date,
  VendorID AS Vendor_ID,
  COUNT(*) AS Total_Trips,
  ROUND(SUM(total_amount), 2) AS Total_Revenue,
  ROUND(AVG(fare_amount), 2) AS Average_Fare,
  ROUND(AVG(tip_amount), 2) AS Average_Tip
FROM STREAM(LIVE.silver_yellowTaxi)
GROUP BY 
  tpep_pickup_datetime,
  VendorID;